In [ ]:
#%pip install ras-commander pandas requests pathlib matplotlib

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ras_commander import RasExamples, init_ras_project, RasCommander, RasPlan, RasGeo

# Step 1: Initialize RasExamples and extract the Muncie project
ras_examples = RasExamples()
ras_examples.extract_project(["Muncie"])

# Use Path.cwd() to get the current working directory in a Jupyter Notebook
current_directory = Path.cwd()
project_path = current_directory / "example_projects" / "Muncie"

# Step 2: Initialize the Muncie Project using init_ras_project (from ras_commander)
muncie_project = init_ras_project(project_path, "6.5")

# Step 3: Initialize a DataFrame to store execution results
results = []

# Step 4: Iterate over each plan and core count
for plan_number in muncie_project.plan_df['plan_number'].unique():
    print(f"Running sensitivity analysis for Plan {plan_number}")
    
    # Clear geompre files before running the plan
    plan_path = RasPlan.get_plan_path(plan_number)
    RasGeo.clear_geompre_files(plan_path)
    
    for cores in range(1, 9):
        # Set core count for this plan
        RasPlan.set_num_cores(plan_number, cores)
        
        # Time the execution of the plan
        start_time = time.time()
        RasCommander.compute_plan(plan_number)
        execution_time = time.time() - start_time
        
        # Store the results
        results.append({
            "plan_number": plan_number,
            "cores": cores,
            "execution_time": execution_time
        })




In [ ]:
# Step 5: Convert results into a DataFrame
results_df = pd.DataFrame(results)

# Step 6: Extract plan title, shortid, and description
plan_info_df = muncie_project.plan_df[['plan_number', 'title', 'shortid', 'description']]

# Merge the execution results with the plan information
merged_df = pd.merge(results_df, plan_info_df, on='plan_number')

# Step 7: Calculate unit runtime (based on 1 core execution time)
merged_df['unit_runtime'] = merged_df.groupby('plan_number')['execution_time'].transform(lambda x: x / x.iloc[0])

# Display the results dataframe for verification
print("merged_df DataFrame:")
print(merged_df)

# Step 8: Plot a line chart for unit runtime vs. cores for each plan
plt.figure(figsize=(10, 6))
for plan in merged_df['plan_number'].unique():
    plan_data = merged_df[merged_df['plan_number'] == plan]
    plt.plot(plan_data['cores'], plan_data['unit_runtime'], label=f"Plan {plan}")

plt.xlabel("Number of Cores")
plt.ylabel("Unit Runtime (Relative to 1 Core)")
plt.title("Core Count Sensitivity Analysis")
plt.legend(title="Plan Title")
plt.grid(True)
plt.show()